# Классификатор купит / не купит в течение трех дней

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Загрузка данных
def load_data():
    catalog = pd.read_parquet('C:\\Users\\nikit\\Hackaton\\stokman_catalog_preprocessed.pq')
    actions = pd.read_parquet('C:\\Users\\nikit\\Hackaton\\train_actions.pq')
    vector_mapping = pd.read_parquet('C:\\Users\\nikit\\Hackaton\\catalog_vector_mapping.pq')
    vectors = np.load('C:\\Users\\nikit\\Hackaton\\vectors.npz')['arr_0']  # Извлечение эмбеддингов товаров

    return catalog, actions, vector_mapping, vectors

# Предобработка данных
def preprocess_data(actions, catalog, vector_mapping):
    # Преобразование даты
    actions['date'] = pd.to_datetime(actions['date'])
    
    # Присоединение каталога товаров
    actions = actions.explode('products')  # Распаковка массива products
    actions = actions.rename(columns={'products': 'product_id'})
    actions = actions.merge(catalog[['product_id', 'price', 'category_id']], on='product_id', how='left')
    
    # Присоединение векторов товаров
    actions = actions.merge(vector_mapping, on='product_id', how='left')
    
    return actions

if __name__ == "__main__":
    catalog, actions, vector_mapping, vectors = load_data()
    actions = preprocess_data(actions, catalog, vector_mapping)
    actions.to_parquet('actions_preprocessed.pq')

In [3]:
import pandas as pd

# Генерация временных и взаимодействующих признаков
def generate_features(actions):
    # Время с последнего действия для каждого пользователя
    actions['days_since_last_action'] = actions.groupby('user_id')['date'].diff().dt.days
    
    # Признаки активности за последние 3 дня
    def count_recent_actions(df, days):
        recent = df[df['date'] >= df['date'].max() - pd.Timedelta(days=days)]
        return recent.groupby('user_id')['action'].count()
    
    recent_activity_3d = count_recent_actions(actions, 3)
    
    # Количество покупок, добавлений в корзину и просмотров
    agg_features = actions.groupby('user_id').agg({
        'product_id': 'nunique',  # Количество уникальных товаров
        'price': 'mean',          # Средняя цена товаров
        'action': ['count', lambda x: (x == 5).sum()],  # Количество действий и покупок
        'days_since_last_action': 'min'  # Время с последнего действия
    }).reset_index()
    
    agg_features.columns = ['user_id', 'n_unique_products', 'avg_price', 'n_actions', 'n_orders', 'days_since_last_action']
    
    # Объединение с активностью за 3 дня
    agg_features = agg_features.merge(recent_activity_3d, on='user_id', how='left')
    agg_features = agg_features.rename(columns={'action': 'actions_last_3d'})
    
    return agg_features

if __name__ == "__main__":
    actions = pd.read_parquet('actions_preprocessed.pq')
    user_features = generate_features(actions)
    
    user_features.to_parquet('user_features.pq')

In [4]:
# Сгенерированные фичи для предсказания покупки:
# Средняя цена интересующих товаров - avg_price
# Время с последней активности - days_since_last_action
# Общее количество действий - n_actions
# Количество покупок - n_orders
# Количество уникальных товаров - n_unique_products
# Действия за последние три дня - actions_last_3d

In [5]:
import pandas as pd

#Предобработка отсутствующих значений
def null_data_preprocessing(user_features):
    user_features.fillna(0)
    user_features.to_parquet('user_features.pq')

In [6]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score
# import mlflow
# import mlflow.lightgbm

# Загрузка данных
def load_features_data():
    features = pd.read_parquet('user_features.pq')
    return features

# Обучение модели
def train_model(features):
    # Целевая переменная: наличие заказов   
    X = features.drop(columns=['n_orders'])  # Признаки
    y = (features['n_orders'] > 0).astype(int)  # Купил ли товар
    
    # Разделение на тренировочные и тестовые данные
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_test_save = X_test.copy(deep=True)
    X_train = X_train.drop('user_id', axis=1)
    X_test = X_test.drop('user_id', axis=1)
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test)
    
    # Параметры модели
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'feature_fraction': 0.9
    }
    
    # # Логирование через MLFlow
    # mlflow.lightgbm.autolog()
    
    # with mlflow.start_run():
    model = lgb.train(params, train_data, valid_sets=[valid_data], num_boost_round=100)
    
    # Предсказания и метрики
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # mlflow.log_metric("precision", precision)
    # mlflow.log_metric("recall", recall)

    print(classification_report(y_test, y_pred))
    print(f'Wirh accuracy: {accuracy_score(y_test, y_pred)}')
    
    return model, X_test, X_test_save

if __name__ == "__main__":
    features = load_features_data()
    null_data_preprocessing(features)
    features = load_features_data()

    
    model, X_test, X_test_user = train_model(features)
    model.save_model('lgb_classifier.txt')

[LightGBM] [Info] Number of positive: 2654, number of negative: 372894
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009736 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 942
[LightGBM] [Info] Number of data points in the train set: 375548, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.007067 -> initscore=-4.945226
[LightGBM] [Info] Start training from score -4.945226
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     93250
           1       0.51      0.20      0.29       637

    accuracy                           0.99     93887
   macro avg       0.75      0.60      0.64     93887
weighted avg       0.99      0.99      0.99     93887

Wirh accuracy: 0.9932685036267002


In [7]:
#Выбираем 3000 пользователей по наибольшей вероятности
model, X_test, X_test_save = train_model(load_features_data())

X_test_save = X_test_save.reset_index()

X_test['pred'] = model.predict(X_test)
X_test = X_test.reset_index()

merged_users = X_test.merge(X_test_save, on='index')
sorted_users = merged_users.sort_values('pred', ascending=False).head(3000)
sorted_users[['user_id', 'pred']]

[LightGBM] [Info] Number of positive: 2654, number of negative: 372894
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009851 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 942
[LightGBM] [Info] Number of data points in the train set: 375548, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.007067 -> initscore=-4.945226
[LightGBM] [Info] Start training from score -4.945226
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     93250
           1       0.51      0.20      0.29       637

    accuracy                           0.99     93887
   macro avg       0.75      0.60      0.64     93887
weighted avg       0.99      0.99      0.99     93887

Wirh accuracy: 0.9932685036267002


,user_id,pred
50224,a046199a-9f61-11ea-a0d7-002590e45c38,0.893693
81700,4814fc56-f93c-11eb-a6e9-002590c82437,0.886025
74487,81e3ac7e-436d-11ef-86e0-002590c0647c,0.878027
87929,101762d6-7733-11ee-bbb1-002590c82437,0.877128
80924,99f0590a-bea9-11ee-bbb1-002590c82436,0.874505
...,...,...
80705,8f8056b2-2f02-11eb-86e0-002590c0647c,0.053368
62219,e42c4da4-22f0-11ed-a044-002590c82437,0.053361
3019,890e8a10-9a70-11ee-86e0-002590c0647c,0.053286
14130,cfb1e80e-46a5-11ef-86e0-002590c0647c,0.053270


#### Optuna for binary classificator

In [9]:
# import optuna
# import lightgbm as lgb
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score

# # Загрузка данных
# def load_features_data():
#     features = pd.read_parquet('user_features.pq')
#     return features

# # Функция для обучения модели
# def train_model(X_train, X_test, y_train, y_test, trial):
#     # Определение пространства гиперпараметров для оптимизации
#     params = {
#         'objective': 'binary',
#         'metric': 'binary_logloss',
#         'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # Логарифмическая сетка
#         'num_leaves': trial.suggest_int('num_leaves', 20, 100),
#         'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
#         'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'max_depth': trial.suggest_int('max_depth', 3, 12),  # Ограничение по глубине дерева
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
#     }
    
#     # Создание данных для обучения и валидации
#     train_data = lgb.Dataset(X_train, label=y_train)
#     valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
#     # Обучение модели
#     model = lgb.train(params, train_data, valid_sets=[valid_data], num_boost_round=100)
    
#     # Предсказания и метрики
#     y_pred = (model.predict(X_test) > 0.5).astype(int)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
    
#     return precision, recall

# # Функция оптимизации
# def objective(trial):
#     features = load_features_data()
    
#     # Целевая переменная: наличие заказов
#     features = features.drop('user_id', axis=1)
#     X = features.drop(columns=['n_orders'])  # Признаки
#     y = (features['n_orders'] > 0).astype(int)  # Купил ли товар
    
#     # Разделение на тренировочные и тестовые данные
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     # Обучение модели с текущими гиперпараметрами
#     precision, recall = train_model(X_train, X_test, y_train, y_test, trial)
    
#     # Оптимизируем по F1-мере (или precision/recall, в зависимости от задачи)
#     f1_score = 2 * (precision * recall) / (precision + recall)
    
#     return f1_score

# # Оптимизация гиперпараметров с использованием Optuna
# def optimize_params():
#     # Создание объекта оптимизации
#     study = optuna.create_study(direction='maximize')  # Мы максимизируем F1-меру
#     study.optimize(objective, n_trials=1000)  # Оптимизируем на 50 итерациях (можно увеличить для лучшего результата)
    
#     print(f"Best trial: {study.best_trial.params}")
#     return study.best_trial.params

# if __name__ == "__main__":
#     # Поиск оптимальных гиперпараметров
#     best_params = optimize_params()
    
#     # Загрузка данных с уже найденными оптимальными параметрами
#     features = load_features_data()
    
#     # Разделение на признаки и целевую переменную
#     features = features.drop('user_id', axis=1)
#     X = features.drop(columns=['n_orders'])
#     y = (features['n_orders'] > 0).astype(int)
    
#     # Разделение на тренировочные и тестовые данные
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     # Использование оптимальных параметров для обучения модели
#     model = lgb.train(best_params, lgb.Dataset(X_train, label=y_train), valid_sets=[lgb.Dataset(X_test, label=y_test)], num_boost_round=100)
    
#     # Предсказания и вывод метрик
#     y_pred = (model.predict(X_test) > 0.5).astype(int)
#     print(classification_report(y_test, y_pred))
#     print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    
#     # Сохранение модели
#     model.save_model('optimized_lgb_classifier.txt')

# Модель ранжирования

In [11]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score, precision_score, recall_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder
import mlflow

In [12]:
# Загрузка данных
catalog = pd.read_parquet('stokman_catalog_preprocessed.pq')
actions = pd.read_parquet('train_actions.pq')
catalog_vector_mapping = pd.read_parquet('catalog_vector_mapping.pq')
vectors = np.load('vectors.npz')['arr_0']  # Извлечение эмбеддингов товаров

# Преобразование даты в формате datetime
actions['date'] = pd.to_datetime(actions['date'])
catalog['add_date'] = pd.to_datetime(catalog['add_date'])

# Пример слияния данных по product_id
catalog = catalog.merge(catalog_vector_mapping, on='product_id', how='left')

In [13]:
products_counter = {}
for i in tqdm(range(0, actions.shape[0])):
    prod_array = actions.iat[i, 4]
    for i in prod_array:
        if i in products_counter:
            products_counter[i] += 1
        else:
            products_counter[i] = 1

100%|██████████| 6580936/6580936 [01:02<00:00, 104953.82it/s]


In [14]:
def popularity(x):
    try:
        return round(products_counter[x] / actions.shape[0] * 10**5, 2)
    except Exception:
        return None

In [15]:
def future_generation(catalog_data):
    catalog_data['price_diff'] = 1 - catalog_data['price'] / catalog_data['old_price']
    catalog_data['popularity'] = catalog_data['product_id'].apply(popularity)
    return catalog_data

In [16]:
catalog = future_generation(catalog)

In [17]:
# Развернем массивы product_id в отдельные строки
actions = actions.explode('products')

# Переименуем колонку для удобства
actions = actions.rename(columns={'products': 'product_id'})

user_item_interactions = actions.groupby(['user_id', 'product_id']).agg(
    views=('action', lambda x: (x == 0).sum()),
    likes=('action', lambda x: (x == 1).sum()),
    add_to_cart=('action', lambda x: (x == 2).sum()),
    orders=('action', lambda x: (x == 5).sum()),
    last_action_time=('date', 'max')
).reset_index()

# Считаем общее количество действий пользователя
user_features = actions.groupby('user_id').agg(
    total_actions=('action', 'count'),
    total_orders=('action', lambda x: (x == 5).sum()),
    total_views=('action', lambda x: (x == 0).sum())
).reset_index()

# Признаки товара
item_features = actions.groupby('product_id').agg(
    total_views=('action', lambda x: (x == 0).sum()),
    total_add_to_cart=('action', lambda x: (x == 2).sum()),
    total_orders=('action', lambda x: (x == 5).sum())
).reset_index()

# Объединяем пользовательские и товарные признаки
data = user_item_interactions.merge(user_features, on='user_id', how='left')
data = data.merge(item_features, on='product_id', how='left')
data = data.merge(catalog[['product_id', 'category_id', 'price', 'old_price', 'price_diff', 'popularity']], on='product_id', how='left')


In [18]:
# Предположим, что у нас есть временной промежуток для обучения и тестирования
train_end_date = pd.Timestamp('2024-09-21')
test_start_date = pd.Timestamp('2024-09-22')

# Делаем таргет (покупка или нет) на основе данных о заказах (action == 5)
train_data = data[data['last_action_time'] <= train_end_date]
train_data['target'] = (train_data['orders'] > 0).astype(int)

# Тестовая выборка
test_data = data[data['last_action_time'] >= test_start_date]
test_data['target'] = (test_data['orders'] > 0).astype(int)

C:\Users\nikit\AppData\Local\Temp\ipykernel_18712\3038899646.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['target'] = (train_data['orders'] > 0).astype(int)
C:\Users\nikit\AppData\Local\Temp\ipykernel_18712\3038899646.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['target'] = (test_data['orders'] > 0).astype(int)


In [19]:
#Присоединяем эмбеддинги товаров
catalog_vector_mapping = catalog_vector_mapping.set_index('vector_id')
vectors = pd.DataFrame(vectors)
catalog_mapping = catalog_vector_mapping.merge(vectors, left_index=True, right_index=True)

train_data = train_data.merge(catalog_mapping, on='product_id', how='left')
test_data = test_data.merge(catalog_mapping, on='product_id', how='left')

In [20]:
# Подготовка данных для обучения
train_data = pd.read_parquet('train_data.pq')
test_data = pd.read_parquet('test_data.pq')

train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

train_data['category_id'] = train_data['category_id'].astype(int) 
test_data['category_id'] = test_data['category_id'].astype(int)

# Подготовим данные для обучения
X = train_data.drop(columns=['product_id', 'last_action_time', 'orders', 'target'])  # Удаляем также 'orders' и 'target'
y = train_data['target']  # Используем целевую переменную

# Получаем список пользователей
groups = train_data['user_id']

# Создаем объект GroupShuffleSplit для корректного разбиения данных по пользователям
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Разделяем данные на тренировочный и валидационный наборы, гарантируя, что данные одного пользователя будут в одном наборе
train_idx, val_idx = next(gss.split(X, y, groups=groups))

# Подготовим тренировочные и валидационные данные
X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

# Считаем размер групп для каждого набора
train_groups = X_train.groupby('user_id').size().values
val_groups = X_val.groupby('user_id').size().values

X_train = X_train.drop('user_id', axis=1)
X_val = X_val.drop('user_id', axis=1)
# Создаем обучающие и валидационные наборы для LightGBM
train_data = lgb.Dataset(X_train, label=y_train, group=train_groups)
val_data = lgb.Dataset(X_val, label=y_val, group=val_groups, reference=train_data)

# Параметры для модели LambdaRank
from sklearn.metrics import precision_score, recall_score

# Параметры модели
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'learning_rate': 0.07,
    'num_leaves': 49,
    'min_data_in_leaf': 84,
    'feature_fraction': 0.95,
    'bagging_fraction': 0.93,
    'bagging_freq': 3,
    'lambda_l1': 0.0129,
    'lambda_l2': 0.000137
}

# Начало трекинга
with mlflow.start_run():
    # Логирование параметров
    mlflow.log_params(params)

    # Обучение модели
    ranker = lgb.train(
        params,
        train_data,
        num_boost_round=100,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'valid'],
    )

    # Логирование модели
    mlflow.lightgbm.log_model(ranker, artifact_path="lgbm_ranking_model", registered_model_name='ranked_model')

    # Предсказания на валидационных данных
    y_val_pred = ranker.predict(X_val)

    # Преобразуем предсказания в бинарные (используем порог 0.5)
    y_val_pred_binary = (y_val_pred > 0.5).astype(int)

    # Расчет метрик precision и recall на валидационных данных
    precision_val = precision_score(y_val, y_val_pred_binary, average='weighted')
    recall_val = recall_score(y_val, y_val_pred_binary, average='weighted')

    # Логирование precision и recall на валидационных данных в MLflow
    mlflow.log_metric("precision_val", precision_val)
    mlflow.log_metric("recall_val", recall_val)

    print(f'Validation Precision: {precision_val:.4f}')
    print(f'Validation Recall: {recall_val:.4f}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.416379 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100033
[LightGBM] [Info] Number of data points in the train set: 370637, number of used features: 398


2024/10/12 22:26:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ranked_model' already exists. Creating a new version of this model...
Created version '5' of model 'ranked_model'.


Validation Precision: 0.9918
Validation Recall: 0.9873


In [22]:
from sklearn.metrics import ndcg_score, precision_score, recall_score
import numpy as np

test_data = pd.read_parquet('test_data.pq')
test_data = test_data.fillna(0)
test_data['category_id'] = test_data['category_id'].astype(int)

# Функция для расчета NDCG для каждой группы
def calculate_ndcg(test_data, k):
    ndcg_scores = []
    # Для каждого пользователя
    for user_id, group in test_data.groupby('user_id'):
        y_true = group['target'].values  # истинные значения (релевантность)
        y_score = group['pred'].values  # предсказанные оценки
        if len(y_true) > 1:  # NDCG имеет смысл только для группы из нескольких товаров
            ndcg = ndcg_score([y_true], [y_score], k=k)
            ndcg_scores.append(ndcg)
    return np.mean(ndcg_scores)

# Функция для расчета Precision@K и Recall@K
def precision_recall_at_k(test_data, k):
    precision_scores = []
    recall_scores = []
    # Для каждого пользователя
    for user_id, group in test_data.groupby('user_id'):
        # Сортируем по предсказанным оценкам
        group = group.sort_values('pred', ascending=False)
        y_true = group['target'].values  # истинные метки
        y_pred = group['pred'].values  # предсказанные метки

        # Рассматриваем топ-K
        top_k = y_true[:k]
        relevant_items = np.sum(y_true)  # общее количество релевантных товаров

        precision = np.sum(top_k) / k  # Precision@K
        recall = np.sum(top_k) / relevant_items if relevant_items > 0 else 0  # Recall@K

        precision_scores.append(precision)
        recall_scores.append(recall)

    return np.mean(precision_scores), np.mean(recall_scores)

# Логирование NDCG@K, Precision@K и Recall@K
K = 25  # Топ-K для Precision и Recall

# Подготовка тестовых данных
X_test = test_data.drop(columns=['product_id', 'last_action_time', 'orders', 'target', 'user_id'])  # Удаляем лишние столбцы
y_test = test_data['target']  # Истинные метки для тестовых данных

# Предсказания для тестового набора
test_data['pred'] = ranker.predict(X_test)

# Вычисление NDCG@K
mean_ndcg = calculate_ndcg(test_data, k=K)
print(f'NDCG@{K}: {mean_ndcg:.4f}')

# Вычисление Precision@K и Recall@K
mean_precision, mean_recall = precision_recall_at_k(test_data, k=K)
print(f'Precision@{K}: {mean_precision:.4f}')
print(f'Recall@{K}: {mean_recall:.4f}')


NDCG@25: 0.0456
Precision@25: 0.0020
Recall@25: 0.0193


In [23]:
import pandas as pd
import numpy as np

# Предполагается, что модель ranker уже обучена и тестовые данные загружены в test_data
# Идентификаторы пользователей и товаров также содержатся в test_data

# Прогнозирование вероятности для тестового набора данных
def get_top_25_relevant_products(test_data, model):
    # Предсказания для тестового набора
    X_test = test_data.drop(columns=['user_id', 'product_id', 'last_action_time', 'orders', 'target'])
    test_data['pred'] = model.predict(X_test)
    
    # Сортируем данные по 'user_id' и предсказанным значениям ('pred')
    test_data_sorted = test_data.sort_values(by=['user_id', 'pred'], ascending=[True, False])
    
    # Для каждого пользователя выбираем топ-25 товаров
    top_25 = test_data_sorted.groupby('user_id').head(25)

    # Создаем новый датафрейм с 'user_id' и списком 25 наиболее релевантных 'product_id'
    top_25_relevant_products = top_25.groupby('user_id').agg({'product_id': lambda x: list(x)})
    
    return top_25_relevant_products

# Пример вызова функции
test_data = pd.read_parquet('test_data.pq')
test_data = test_data.fillna(0)
test_data['category_id'] = test_data['category_id'].astype(int)

top_25_products_per_user = get_top_25_relevant_products(test_data, ranker)

# Выводим результаты
top_25_products_per_user.head()

,product_id
user_id,
0,"[5168833, 5168824, 5168806]"
1,"[5422963, 6612845, 6715845, 6538627, 6604130, ..."
2,[6274777]
3,[6078485]
4,[6269449]


In [ ]:
# import optuna
# import lightgbm as lgb
# import numpy as np
# from sklearn.model_selection import GroupShuffleSplit
# from sklearn.metrics import ndcg_score

# # Функция для обучения и оценки модели
# def objective(trial):
#     train_data = pd.read_parquet('train_data.pq')
#     test_data = pd.read_parquet('test_data.pq')
    
#     train_data = train_data.fillna(0)
#     test_data = test_data.fillna(0)
    
#     train_data['category_id'] = train_data['category_id'].astype(int) 
#     test_data['category_id'] = test_data['category_id'].astype(int)
#     # Гиперпараметры, которые мы будем оптимизировать
#     params = {
#         'objective': 'lambdarank',
#         'metric': 'ndcg',
#         'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True), 
#         'num_leaves': trial.suggest_int('num_leaves', 20, 100),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'lambda_l1': trial.suggest_float('lambda_l1', 1e-4, 1.0, log=True),
#         'lambda_l2': trial.suggest_float('lambda_l2', 1e-4, 1.0, log=True)
#     }

#     # Разделяем данные на тренировочные и валидационные
#     gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#     train_idx, val_idx = next(gss.split(X, y, groups=groups))

#     X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     train_groups = X_train.groupby('user_id').size().values
#     val_groups = X_val.groupby('user_id').size().values

#     # Создаем обучающие и валидационные наборы для LightGBM
#     train_data = lgb.Dataset(X_train, label=y_train, group=train_groups)
#     val_data = lgb.Dataset(X_val, label=y_val, group=val_groups, reference=train_data)

#     # Обучение модели
#     ranker = lgb.train(
#         params,
#         train_data,
#         num_boost_round=100,
#         valid_sets=[train_data, val_data],
#         valid_names=['train', 'valid'],
#     )
    
#     K = 25  # Топ-K для Precision и Recall

#     # Подготовка тестовых данных
#     X_test = test_data.drop(columns=['product_id', 'last_action_time', 'orders', 'target'])  # Удаляем лишние столбцы
#     y_test = test_data['target']  # Истинные метки для тестовых данных
    
#     # Предсказания для тестового набора
#     test_data['pred'] = ranker.predict(X_test)
    
#     # Вычисление NDCG@K
#     mean_ndcg = calculate_ndcg(test_data, k=K)
#     print(f'NDCG@{K}: {mean_ndcg:.4f}')

#     return mean_ndcg
    

# # Создание объекта исследования Optuna
# study = optuna.create_study(direction='maximize')  # Максимизируем NDCG
# study.optimize(objective, n_trials=50)  # Запускаем 50 испытаний

# # Выводим результаты
# print("Best hyperparameters: ", study.best_params)
# print("Best Recall: ", study.best_value)

# ##В качестве метрики сделать массив со всеми купленными пользователем товарами, так же седалть массив с топ 25 товарами по предсказанию 
# ##По этим двум столбцам считать recall типо сколько купленных товаров в нашем предсказанном массиве и делаить на общее число покупок

In [ ]:
# class MultiColumnLabelEncoder:

#     def __init__(self, columns=None):
#         self.columns = columns # array of column names to encode


#     def fit(self, X, y=None):
#         self.encoders = {}
#         columns = X.columns if self.columns is None else self.columns
#         for col in columns:
#             self.encoders[col] = LabelEncoder().fit(X[col])
#         return self


#     def transform(self, X):
#         output = X.copy()
#         columns = X.columns if self.columns is None else self.columns
#         for col in columns:
#             output[col] = self.encoders[col].transform(X[col])
#         return output


#     def fit_transform(self, X, y=None):
#         return self.fit(X,y).transform(X)


#     def inverse_transform(self, X):
#         output = X.copy()
#         columns = X.columns if self.columns is None else self.columns
#         for col in columns:
#             output[col] = self.encoders[col].inverse_transform(X[col])
#         return output

In [ ]:
# columns_to_encode = ['user_id']

# train_encoder = MultiColumnLabelEncoder(columns_to_encode)
# train_data = train_encoder.fit_transform(train_data)

# test_encoder = MultiColumnLabelEncoder(columns_to_encode)
# test_data = test_encoder.fit_transform(test_data)

In [ ]:
# # Подготовка тестовых данных
#     X_test = test_data.drop(columns=['product_id', 'last_action_time', 'orders', 'target'])
#     test_data['pred'] = ranker.predict(X_test)

#     # Ранжирование товаров для каждого пользователя
#     test_data_sorted = test_data.sort_values(by=['user_id', 'pred'], ascending=False)
#     top_25 = test_data_sorted.groupby('user_id').head(25)

#     # Группировка и сохранение результатов
#     result = top_25.groupby('user_id').agg({
#         'product_id': lambda x: list(x)
#     }).reset_index()

#     purchased_data = test_data[test_data['orders'] > 0]
#     purchased_products = purchased_data.groupby('user_id').agg({
#         'product_id': lambda x: list(x)
#     }).reset_index()

#     table = result.merge(purchased_products, on='user_id', how='left')
#     table = table.fillna(0)

#     # Функция для расчета recall
#     def recall(table):
#         recall_values = []
#         for i in tqdm(range(0, table.shape[0])):
#             preds = table.iat[i, 1]
#             buys = table.iat[i, 2]
#             if buys == 0:
#                 continue
#             intersect = list(set(preds) & set(buys))
#             recall_values.append(len(intersect) / len(buys))

#         return np.mean(recall_values)

#     # Расчет метрики recall на тестовых данных
#     recall_test = recall(table)

#     # Логирование метрики recall на тестовых данных в MLflow
#     mlflow.log_metric("recall_test", recall_test)

#     print(f'Test Recall: {recall_test:.4f}')

# MLFlowTesting

In [ ]:
from mlflow.tracking import MlflowClient

# Инициализируем клиента MLflow
client = MlflowClient()

# Название вашей модели (замените 'ranked_model' на нужное имя)
model_name = "ranked_model"

# Получаем информацию обо всех версиях модели
model_versions = client.get_latest_versions(model_name)

# Выводим информацию о стадиях каждой версии модели
for version in model_versions:
    print(f"Version: {version.version}, Stage: {version.current_stage}, Status: {version.status}")


In [ ]:
from mlflow.tracking import MlflowClient

# Инициализируем клиента MLflow
client = MlflowClient()

# Название вашей модели
model_name = "ranked_model"

# Версия модели, которую нужно перевести в новую стадию (укажите версию вашей модели)
model_version = 1  # Замените на актуальную версию

# Новая стадия (например, "Staging" или "Production")
new_stage = "Production"  # Может быть "Staging", "Production", или "Archived"

# Переводим модель в новую стадию
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage
)

print(f"Модель {model_name} версии {model_version} переведена в стадию {new_stage}")


In [ ]:
import mlflow
import mlflow.lightgbm
import pandas as pd

# URI модели из MLflow Model Registry (например, "models:/<model_name>/Production")
model_uri = "models:/ranked_model/Production"  # Замените на своё имя модели

# Загрузка модели из MLflow
model = mlflow.lightgbm.load_model(model_uri)

# Подготовка тестового набора данных
test_data = pd.read_parquet('test_data.pq')
test_data = test_data.fillna(0)
test_data['category_id'] = test_data['category_id'].astype(int)
X_test = test_data.drop(columns=['product_id', 'last_action_time', 'orders', 'target'])

# Использование загруженной модели для предсказания на тестовых данных
test_data['pred'] = model.predict(X_test)

# Ранжирование товаров для каждого пользователя по предсказанным оценкам
test_data_sorted = test_data.sort_values(by=['user_id', 'pred'], ascending=False)

# Выбираем топ-25 товаров для каждого пользователя
top_25 = test_data_sorted.groupby('user_id').head(25)

# Преобразуем товары в массив для каждого пользователя
result = top_25.groupby('user_id').agg({
    'product_id': lambda x: list(x),  # Преобразуем товары в список
    'pred': lambda x: list(x)  # Преобразуем предсказанные оценки в список (опционально)
}).reset_index()

# Фильтрация только купленных товаров (предполагается, что 'orders' > 0 означает покупку)
purchased_data = test_data[test_data['orders'] > 0]

# Группировка по user_id и преобразование списка купленных товаров в массив
purchased_products = purchased_data.groupby('user_id').agg({
    'product_id': lambda x: list(x)  # Преобразуем купленные товары в список
}).reset_index()

# Выводим результат
purchased_products.columns = ['user_id', 'bought_products']

# Объединение результатов
table = result.merge(purchased_products, on='user_id', how='left')
table = table.fillna(0).drop('pred', axis=1)

# Функция для расчета recall
def recall(table):
    recall_values = []
    for i in tqdm(range(0, table.shape[0])):
        preds = table.iat[i, 1]
        buys = table.iat[i, 2]
        if buys == 0:
            continue
        intersect = list(set(preds) & set(buys))
        recall_values.append(len(intersect) / len(buys))

    return np.mean(recall_values)

# Расчет recall на тестовых данных
recall_test = recall(table)

print(f'Test Recall: {recall_test:.4f}')


In [ ]:
# # Предсказания на валидационных данных
# y_val_pred = ranker.predict(X_val)

# # Преобразуем предсказания в бинарный формат
# y_val_pred_binary = (y_val_pred > 0.5).astype(int)

# # Расчет метрик точности
# # Поскольку у нас многоклассовый случай, выбираем 'weighted'
# precision = precision_score(y_val, y_val_pred_binary, average='weighted')
# recall = recall_score(y_val, y_val_pred_binary, average='weighted')

# print(f'Precision: {precision:.4f}')
# print(f'Recall: {recall:.4f}')

In [ ]:
# # Подготовка тестового набора
# X_test = test_data.drop(columns=['product_id', 'last_action_time', 'orders', 'target'])

# # Предсказания для тестового набора
# test_data['pred'] = ranker.predict(X_test)

# # Ранжируем товары для каждого пользователя по предсказанным оценкам
# test_data_sorted = test_data.sort_values(by=['user_id', 'pred'], ascending=False)

# # Выбираем топ-25 товаров для каждого пользователя
# top_25 = test_data_sorted.groupby('user_id').head(25)

# # Преобразуем товары в массив для каждого пользователя
# result = top_25.groupby('user_id').agg({
#     'product_id': lambda x: list(x),  # Преобразуем товары в список
#     'pred': lambda x: list(x)  # Преобразуем предсказанные оценки в список (опционально)
# }).reset_index()

In [ ]:
# # Фильтрация только купленных товаров (предполагается, что 'orders' > 0 означает покупку)
# purchased_data = test_data[test_data['orders'] > 0]

# # Группировка по user_id и преобразование списка купленных товаров в массив
# purchased_products = purchased_data.groupby('user_id').agg({
#     'product_id': lambda x: list(x)  # Преобразуем купленные товары в список
# }).reset_index()

# # Выводим результат
# purchased_products.columns = ['user_id', 'bought_products']

In [ ]:
# table = result.merge(purchased_products, on='user_id', how='left')
# table = table.fillna(0).drop('pred', axis=1)

In [ ]:
# def recall(table):
#     recall = []
#     for i in tqdm(range(0, table.shape[0])):
#         preds = table.iat[i, 1]
#         buys = table.iat[i, 2]
#         if buys == 0:
#             pass
#         else:
#             intersect = list(set(preds) & set(buys))
#             recall.append(len(intersect) / len(buys))
    
    
#     return np.mean(recall)    

In [ ]:
# score = recall(table)

In [ ]:
# score

In [ ]:
# import optuna
# import lightgbm as lgb
# import numpy as np
# from sklearn.model_selection import GroupShuffleSplit
# from sklearn.metrics import ndcg_score

# # Функция для обучения и оценки модели
# def objective(trial):
#     # Гиперпараметры, которые мы будем оптимизировать
#     params = {
#         'objective': 'lambdarank',
#         'metric': 'ndcg',
#         'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True), 
#         'num_leaves': trial.suggest_int('num_leaves', 20, 100),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'lambda_l1': trial.suggest_float('lambda_l1', 1e-4, 1.0, log=True),
#         'lambda_l2': trial.suggest_float('lambda_l2', 1e-4, 1.0, log=True)
#     }

#     # Разделяем данные на тренировочные и валидационные
#     gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#     train_idx, val_idx = next(gss.split(X, y, groups=groups))

#     X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     train_groups = X_train.groupby('user_id').size().values
#     val_groups = X_val.groupby('user_id').size().values

#     # Создаем обучающие и валидационные наборы для LightGBM
#     train_data = lgb.Dataset(X_train, label=y_train, group=train_groups)
#     val_data = lgb.Dataset(X_val, label=y_val, group=val_groups, reference=train_data)

#     # Обучение модели
#     ranker = lgb.train(
#         params,
#         train_data,
#         num_boost_round=100,
#         valid_sets=[train_data, val_data],
#         valid_names=['train', 'valid'],
#     )
    
#     # Предсказания для тестового набора
#     test_data['pred'] = ranker.predict(X_test)
    
#     # Ранжируем товары для каждого пользователя по предсказанным оценкам
#     test_data_sorted = test_data.sort_values(by=['user_id', 'pred'], ascending=False)
    
#     # Выбираем топ-25 товаров для каждого пользователя
#     top_25 = test_data_sorted.groupby('user_id').head(25)
    
#     # Преобразуем товары в массив для каждого пользователя
#     result = top_25.groupby('user_id').agg({
#         'product_id': lambda x: list(x),  # Преобразуем товары в список
#         'pred': lambda x: list(x)  # Преобразуем предсказанные оценки в список (опционально)
#     }).reset_index()

#     table = result.merge(purchased_products, on='user_id', how='left')
#     table = table.fillna(0).drop('pred', axis=1)

#     return recall(table)

    

# # Создание объекта исследования Optuna
# study = optuna.create_study(direction='maximize')  # Максимизируем NDCG
# study.optimize(objective, n_trials=50)  # Запускаем 50 испытаний

# # Выводим результаты
# print("Best hyperparameters: ", study.best_params)
# print("Best Recall: ", study.best_value)

# ##В качестве метрики сделать массив со всеми купленными пользователем товарами, так же седалть массив с топ 25 товарами по предсказанию 
# ##По этим двум столбцам считать recall типо сколько купленных товаров в нашем предсказанном массиве и делаить на общее число покупок

# PipeLine

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import timedelta

# Преобразование даты и извлечение признаков
class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self  # Ничего не нужно обучать

    def transform(self, X):
        # Преобразование даты
        X['date'] = pd.to_datetime(X['date'])

        # Присоединение каталога товаров
        X = X.explode('products')  # Распаковка массива products
        X = X.rename(columns={'products': 'product_id'})
        # Пример присоединения других данных, пока используем фиктивные данные
        X['price'] = np.random.rand(X.shape[0])  # Фиктивные данные
        X['category_id'] = np.random.randint(1, 10, size=X.shape[0])  # Фиктивные данные
        
        return X


# Генерация признаков
class FeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self  # Ничего не нужно обучать

    def transform(self, X):
        # Время с последнего действия для каждого пользователя
        X['days_since_last_action'] = X.groupby('user_id')['date'].diff().dt.days

        # Признаки активности за последние 3 дня
        def count_recent_actions(df, days):
            recent = df[df['date'] >= df['date'].max() - pd.Timedelta(days=days)]
            return recent.groupby('user_id')['action'].count()

        recent_activity_3d = count_recent_actions(X, 3)

        # Количество покупок, добавлений в корзину и просмотров
        agg_features = X.groupby('user_id').agg({
            'product_id': 'nunique',  # Количество уникальных товаров
            'price': 'mean',          # Средняя цена товаров
            'action': ['count', lambda x: (x == 5).sum()],  # Количество действий и покупок
            'days_since_last_action': 'min'  # Время с последнего действия
        }).reset_index()

        agg_features.columns = ['user_id', 'n_unique_products', 'avg_price', 'n_actions', 'n_orders', 'days_since_last_action']

        # Объединение с активностью за 3 дня
        agg_features = agg_features.merge(recent_activity_3d, on='user_id', how='left')
        agg_features = agg_features.rename(columns={'action': 'actions_last_3d'})

        return agg_features


# Преобразователь для работы с отсутствующими значениями
class NullDataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self  # Ничего не нужно обучать

    def transform(self, X):
        return X.fillna(0)  # Заполнение пропущенных значений нулями


In [ ]:
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score

# Обучение модели LightGBM
class LightGBMClassifier(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', learning_rate=0.05, num_leaves=31)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def predict_proba(self, X):
        return self.model.predict_proba(X)


# Создание пайплайна
pipeline = Pipeline(steps=[
    ('preprocessor', DataPreprocessor()),  # Предобработка данных
    ('feature_generator', FeatureGenerator()),  # Генерация признаков
    ('null_data', NullDataPreprocessor()),  # Предобработка пропущенных данных
    ('classifier', LightGBMClassifier())  # Классификатор LightGBM
])

# Загрузка данных
def load_features_data():
    features = pd.read_parquet('user_features.pq')
    return features

if __name__ == "__main__":
    features = load_features_data()

    X = features.drop(columns=['n_orders', 'user_id'])  # Признаки
    y = (features['n_orders'] > 0).astype(int)  # Целевая переменная

    # Разделение на тренировочные и тестовые данные
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Обучение модели
    pipeline.fit(X_train, y_train)

    # Предсказания
    y_pred = pipeline.predict(X_test)

    # Оценка качества модели
    print(classification_report(y_test, y_pred))
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')